In [ ]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

In [3]:
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool
from openai import OpenAI
import os

# Directly set your OpenAI and Serper API keys here
openai_api_key = "your-api-key"
serper_api_key = "your-api-key" # you can find it here https://serper.dev/api-key, don't worry, it has enough free credits.

# Set the environment variables
os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4-turbo'
os.environ["SERPER_API_KEY"] = serper_api_key

# Initialize the OpenAI client
client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])

# Define the Input Parsing Agent
input_parser = Agent(
    role='Input Parsing Specialist',
    goal='Parse the user\'s input to extract job {role}, {location}, {company_type}, and {preferences}.',
    verbose=True,
    memory=False,
    backstory=(
        "You are a linguistic expert with an exceptional ability to interpret and extract key details from text inputs."
        " Your sharp understanding of language nuances allows you to parse user prompts accurately to provide clear and actionable data."
    ),
    allow_delegation=False
)


# Define the Input Parsing Task
def parse_user_input(prompt):
    print(f"Received prompt: {prompt}")  # Debugging line
    response = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=[
            {"role": "system", "content": "You are an expert in extracting information from user prompts."},
            {"role": "user", "content": f"Extract the following details from the user's prompt: job role, location, company type, and any other specified preferences.\n\nUser Prompt: {prompt}\n\nExpected Output: A JSON object with the following keys: role, location, company_type, preferences.\nExample Output: {{'role': 'Data Analyst', 'location': 'Bengaluru', 'company_type': 'Fintech', 'preferences': {{'experience': '5 years'}}}}"}
        ]
    )
    output = response.choices[0].message.content.strip()
    print(f"Parsed output: {output}")  # Debugging line
    return eval(output)  # Note: In a production environment, use a safer method to parse the output

parsing_task = Task(
    description=(
        "Parse the user's prompt to extract the job {role}, {location}, {company_type}, and any other specified {preferences}."
        " Ensure that the extracted information is clear and formatted appropriately for further processing."
    ),
    expected_output='A dictionary with keys: role, location, company_type, and preferences.',
    agent=input_parser,
    run_function=lambda inputs: parse_user_input(inputs['user_prompt'])  # Ensure the function uses the correct input
)


# Define the Job Researcher Agent
job_researcher = Agent(
    role='Senior Job Researcher',
    goal='Identify and compile a list of job postings that match the user\'s specific criteria.',
    verbose=True,
    memory=True,
    backstory=(
        "You have spent years in the field of recruitment, specializing in finding the best job opportunities for clients."
        " Your expertise lies in navigating various job boards, company websites, and social media to uncover hidden gems in the job market."
        " Your dedication and sharp eye for detail ensure that you never miss a relevant job posting."
    ),
    tools=[SerperDevTool()],
    allow_delegation=True
)

# Define the Job Analysis Agent
job_analyst = Agent(
    role='Expert Job Analyst',
    goal='Evaluate and filter job postings to ensure they meet the criteria specified by the user.',
    verbose=True,
    memory=True,
    backstory=(
        "With a background in data analysis and a passion for startups, you excel at discerning which job postings truly offer growth opportunities."
        " Your analytical skills and understanding of the startup ecosystem allow you to sift through numerous postings and identify the most promising positions."
        " You take pride in helping clients find jobs that align with their career goals and aspirations."
    ),
    allow_delegation=False
)

# Define the Job Search Task
def search_for_jobs(parsed_data):
    role = parsed_data.get('role')
    location = parsed_data.get('location')
    company_type = parsed_data.get('company_type')
    preferences = parsed_data.get('preferences')
    print(f"Searching for jobs with role: {role}, location: {location}, company_type: {company_type}, preferences: {preferences}")  # Debugging line
    # Here you would add the logic to search for jobs using the SerperDevTool or any other tool.
    return [
        {'title': 'Data Analyst', 'company': 'Fintech XYZ', 'location': 'Bengaluru', 'description': 'Exciting opportunity in a fintech company.'},
        # Add more job listings as needed
    ]

job_search_task = Task(
    description=(
        "Using the parsed criteria from the user's prompt, search for job postings that match the specified job title, company type, and location."
    ),
    expected_output='A list of job postings that match the criteria, including job title, company name, location, and a brief job description.',
    agent=job_researcher,
    run_function=lambda inputs: search_for_jobs(inputs)  # Use the parsed data
)

# Define the Job Analysis Task
def analyze_jobs(job_list):
    print(f"Analyzing jobs: {job_list}")  # Debugging line
    # Here you would add the logic to analyze the jobs and filter out the best ones.
    return job_list

job_analysis_task = Task(
    description=(
        "Analyze the list of job postings provided by the Job Researcher."
    ),
    expected_output='A refined list of job postings that fit the specified criteria, including detailed analysis of each posting with clickable links.',
    agent=job_analyst,
    run_function=lambda inputs: analyze_jobs(inputs['job_list'])  # Use the job list
)

# Create the crew
job_search_crew = Crew(
    agents=[input_parser, job_researcher, job_analyst],
    tasks=[parsing_task, job_search_task, job_analysis_task],
    process=Process.sequential  # Sequential process
)

# User input
user_prompt = "Product Manager in an AI company in India"

# Kickoff the crew with user input
parsed_result = parse_user_input(user_prompt)
print(f"Parsed Result: {parsed_result}")  # Debugging line
print(type(parsed_result))

# Pass the parsed result to the crew kickoff
result = job_search_crew.kickoff(inputs= parsed_result)
print(result)


Received prompt: Product Manager in an AI company in India
Parsed output: {
  "role": "Product Manager",
  "location": "India",
  "company_type": "AI company",
  "preferences": {}
}
Parsed Result: {'role': 'Product Manager', 'location': 'India', 'company_type': 'AI company', 'preferences': {}}
<class 'dict'>


> Entering new CrewAgentExecutor chain...
Thought: I now can give a great answer

Final Answer: {"role": "Product Manager", "location": "India", "company_type": "AI company", "preferences": {}}

> Finished chain.


> Entering new CrewAgentExecutor chain...
Thought: First, I need to understand the user's criteria for the job search fully before I proceed with the internet search. The criteria include the job title "Product Manager", the location "India", and the company type "AI company".

Action: Search the internet
Action Input: {"search_query": "Product Manager AI company job posting India"} 


Search results: Title: 617 Ai product manager jobs in United States | Glassdoor
Link